In [40]:
#packages related to dataframes
import pandas as pd
import numpy as np
from pathlib import Path
import ast

#package needed to assign user a unique id 
import uuid

#package nee
import csv

# Importing helper function from utils folder 
import sys
sys.path.append('Resources/utils')
from user_form_intake import inital_user_intake, get_user_inital_input, portfolio_user_intake, get_user_portfolio_input

# Grabbing the Data Files 

In [11]:
#Grabing the raw stock data and putting in dataframe
raw_stocks_data_to_load =  Path("Resources/Data/Stock_Index_Raw_Data.csv")
raw_stocks_data_df = pd.read_csv(raw_stocks_data_to_load, header=0, parse_dates=True, index_col="Date", infer_datetime_format=True)
raw_stocks_data_df.sort_index(ascending = True, inplace = True)
raw_stocks_data_df.head()

,Company,Ticker,Close_Price,Industry,Sector
Date,,,,,
2023-02-01,Apple,AAPL,145.43,Computer Manufacturing,Technology
2023-02-01,Amazon,AMZN,105.15,Catalog/Specialty Distribution,Consumer Discretionary
2023-02-01,Nvidia,NVDA,209.43,Semiconductors,Technology
2023-02-01,Alphabet,GOOGL,100.43,"Computer Software: Programming, Data Processing",Technology
2023-02-01,Meta,META,153.12,"Computer Software: Programming, Data Processing",Technology


In [12]:
#Grabing the raw stock data and putting in dataframe
user_data_to_load =  Path("Resources/Data/User_Portfolio_Data.csv")
user_data_df = pd.read_csv(user_data_to_load, header=0, parse_dates=True)
user_data_df.sort_index(ascending = True, inplace = True)
user_data_df.head()

,UUID,User_Name,Sharpe_Ratio,Stocks
0,c8b71fc0-a969-4bbb-80f0-8caeedd3cc6a,Test1,4.54450,"['AVGO', 'AAPL', 'MSFT', 'V', 'NVDA']"
1,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,4.43430,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"
2,56a3ba9e-7b6d-4376-a6d8-ffd2f42cee9d,Chris,3.11234,"['AVGO', 'AAPL', 'MSFT', 'XOM']"
3,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,3.11234,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"
4,2526a1f9-0f46-4b73-a0aa-ac9f5920e3d9,JJ,2.11000,"['AMZN', 'AAPL', 'MSFT', 'JPM']"


# User Input for Game of Stocks

In [13]:
# Envoking the user intake form 1. It will store the information in the following variables so it can be used in notebook later.

inital_user_intake()

Welcome to the Game of Stocks! You will be asked a series of questions to get started. 
    You can choose up to 10 stickes to create portfolio. Remember to diversify your portfolio.   


Textarea(value='Input your name', description='First Name:', placeholder='Type something')

Dropdown(description='Amount', index=10, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Choose Stock Amount:'), valu…

Button(description='Submit', style=ButtonStyle())

In [14]:
# Getting the form values that was store in name, num_of_stocks_choosen 
name, num_of_stocks_choosen = get_user_inital_input()

In [15]:
portfolio_user_intake(num_of_stocks_choosen)

Dropdown(description='Stock 1:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Dropdown(description='Stock 2:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Dropdown(description='Stock 3:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Dropdown(description='Stock 4:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Button(description='Submit', style=ButtonStyle())

In [16]:
# getting the user chosen portfolio
unique_user_portfolio_array = get_user_portfolio_input()

In [17]:
unique_user_portfolio_array

['NVDA', 'AMZN', 'AAPL', 'MSFT']

In [18]:
#calculating individual weights since we will assume it will be evenly split
weight = round((1/len(unique_user_portfolio_array)),5)

weight

0.25

In [19]:
#creating the weight array to apply to stocks
weight_array = [weight] * len(unique_user_portfolio_array)
weight_array

[0.25, 0.25, 0.25, 0.25]

# User Portfolio Calculations 

In [20]:
# Get data from file for tickers in unique_user_portfolio_array using ".loc" and ".isin"
portfolio_returns = raw_stocks_data_df.loc[raw_stocks_data_df["Ticker"].isin(unique_user_portfolio_array)]

# Filter columns to get only the Ticker and Close Price and drop null values
portfolio_returns= portfolio_returns[["Ticker","Close_Price"]].dropna()

# Set tickers as columns
portfolio_returns= portfolio_returns.pivot_table("Close_Price","Date","Ticker")

# Calculate daily returns using "pct_change" and drop null values 
portfolio_returns = portfolio_returns.pct_change().dropna()

# Display data
portfolio_returns.head()

Ticker,AAPL,AMZN,MSFT,NVDA
Date,,,,
2023-02-02,0.037063,0.073799,0.046884,0.036575
2023-02-03,0.024400,-0.084315,-0.023621,-0.028053
2023-02-06,-0.017929,-0.011703,-0.006116,-0.000521
2023-02-07,0.019245,-0.000685,0.042022,0.051401
2023-02-08,-0.017653,-0.020174,-0.003102,0.001443


In [21]:
# Create portfolio variable in dataframe with weights using ".dot"
portfolio_returns["Portfolio"] = portfolio_returns.dot(weight_array)
portfolio_returns.head()

Ticker,AAPL,AMZN,MSFT,NVDA,Portfolio
Date,,,,,
2023-02-02,0.037063,0.073799,0.046884,0.036575,0.048580
2023-02-03,0.024400,-0.084315,-0.023621,-0.028053,-0.027897
2023-02-06,-0.017929,-0.011703,-0.006116,-0.000521,-0.009067
2023-02-07,0.019245,-0.000685,0.042022,0.051401,0.027996
2023-02-08,-0.017653,-0.020174,-0.003102,0.001443,-0.009871


In [22]:
# use to store in user portfolio table 
user_sharpe_ratio= (portfolio_returns["Portfolio"].mean() * (252)) / (portfolio_returns["Portfolio"].std() * np.sqrt(252))
user_sharpe_ratio

3.4124308359758553

# Storing User Info In User Portfolio CSV and Updating The Rank 

In [23]:
#getting a uuid 
unique_id = uuid.uuid4()

#converting uuid to string so it can store as a string in csv and easier to use loc
user_uuid_str = str(unique_id)
print("UUID as string:", user_uuid_str)

UUID as string: 9911f3d6-2a1e-4956-b676-d14ddd12fe0e


In [24]:
# Create a list of dictionaries (each dictionary represents a row of data)
user_data = [{'UUID': user_uuid_str,
              'User_Name': name,
              'Sharpe_Ratio': user_sharpe_ratio,
              'Stocks': f'{unique_user_portfolio_array}'}]

# Create the DataFrame from the list of dictionaries
user_portfolio_df = pd.DataFrame(user_data)

In [25]:
# Append the new row to the DataFrame
# user_data_df = user_data_df.append(user_data, ignore_index=True)

# Concatenate the original DataFrame and the new row DataFrame
user_data_df = pd.concat([user_data_df, user_portfolio_df], ignore_index=True)

# Write the updated DataFrame back to the CSV file
user_data_df.to_csv(user_data_to_load, index=False)

#Print so user can see what was added 
print("New User row added to the User User_Portfolio_Data.csv file. The following was added:")
display(user_portfolio_df)

New User row added to the User User_Portfolio_Data.csv file. The following was added:


,UUID,User_Name,Sharpe_Ratio,Stocks
0,9911f3d6-2a1e-4956-b676-d14ddd12fe0e,alex_1,3.412431,"['NVDA', 'AMZN', 'AAPL', 'MSFT']"


In [27]:
#preview top 5 users BEFORE the rank is updated 
user_data_df.head()

,UUID,User_Name,Sharpe_Ratio,Stocks
0,c8b71fc0-a969-4bbb-80f0-8caeedd3cc6a,Test1,4.54450,"['AVGO', 'AAPL', 'MSFT', 'V', 'NVDA']"
1,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,4.43430,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"
2,56a3ba9e-7b6d-4376-a6d8-ffd2f42cee9d,Chris,3.11234,"['AVGO', 'AAPL', 'MSFT', 'XOM']"
3,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,3.11234,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"
4,2526a1f9-0f46-4b73-a0aa-ac9f5920e3d9,JJ,2.11000,"['AMZN', 'AAPL', 'MSFT', 'JPM']"


In [28]:
#Sort user portfolio table to get new ranking. Sort by descending since higher sharpe ratio the better 
sorted_user_data_df = user_data_df.sort_values(by='Sharpe_Ratio', ascending=False)

#Rank AFTER rank is updated  
sorted_user_data_df.head()


,UUID,User_Name,Sharpe_Ratio,Stocks
0,c8b71fc0-a969-4bbb-80f0-8caeedd3cc6a,Test1,4.544500,"['AVGO', 'AAPL', 'MSFT', 'V', 'NVDA']"
1,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,4.434300,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"
6,9911f3d6-2a1e-4956-b676-d14ddd12fe0e,alex_1,3.412431,"['NVDA', 'AMZN', 'AAPL', 'MSFT']"
2,56a3ba9e-7b6d-4376-a6d8-ffd2f42cee9d,Chris,3.112340,"['AVGO', 'AAPL', 'MSFT', 'XOM']"
3,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,3.112340,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"


In [29]:
#Save new rank to user User_Portfolio_Data.csv Write the updated DataFrame back to the CSV file

sorted_user_data_df.to_csv(user_data_to_load, index=False)

#have to re-import the user portfolio data to get updated sorted list 
user_data_df = pd.read_csv(user_data_to_load, header=0, parse_dates=True)
user_data_df.sort_index(ascending = True, inplace = True)

In [30]:
#User Ranking of top 5 AFTER rank is updated  

user_data_df.head()

,UUID,User_Name,Sharpe_Ratio,Stocks
0,c8b71fc0-a969-4bbb-80f0-8caeedd3cc6a,Test1,4.544500,"['AVGO', 'AAPL', 'MSFT', 'V', 'NVDA']"
1,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,4.434300,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"
2,9911f3d6-2a1e-4956-b676-d14ddd12fe0e,alex_1,3.412431,"['NVDA', 'AMZN', 'AAPL', 'MSFT']"
3,56a3ba9e-7b6d-4376-a6d8-ffd2f42cee9d,Chris,3.112340,"['AVGO', 'AAPL', 'MSFT', 'XOM']"
4,951bb78a-dacb-4047-a255-2eb50314a9fd,Amy,3.112340,"['V', 'GOOGL', 'JNJ', 'MSFT', 'XOM']"


In [31]:
# Grab Postition of User Ranking 

condition = user_data_df['UUID'] == user_uuid_str
matching_rows = user_data_df[condition]

# Get the index of the first matching row (if any)
if not matching_rows.empty:
    row_index = matching_rows.index[0]
#     print(f"Index of the row: {row_index}")
    print(f"{name}, your rank is {row_index + 1}" )
else:
    print("Error, user info not found")
    


alex_1, your rank is 3


# User Performance Compared to Other Players Calulation and Market

In [54]:
# Create a dataframe for market indicators and calculate daily returns and drop nulls 
sp_returns = raw_stocks_data_df.loc[raw_stocks_data_df["Ticker"]=="SPX"]
sp_returns = sp_returns[["Close_Price"]].pct_change().dropna()

ndaq_returns = raw_stocks_data_df.loc[raw_stocks_data_df["Ticker"]=="NDAQ"]
ndaq_returns = ndaq_returns[["Close_Price"]].pct_change().dropna()

# Rename columns to SP500 and NDAQ
sp_returns.columns=["SP500"]
ndaq_returns.columns=["NDAQ"]

# Combine dataframes
market_returns = pd.concat([sp_returns, ndaq_returns], axis="columns", join="inner")
market_returns.head()

,SP500,NDAQ
Date,,
2023-02-02,0.014699,0.032540
2023-02-03,-0.010355,-0.015889
2023-02-06,-0.006140,-0.009953
2023-02-07,0.012873,0.019040
2023-02-08,-0.011081,-0.016780


In [55]:
# Create a dataframe for top players and calculate daily returns and drop nulls 
top_1_str_to_list = ast.literal_eval(user_data_df.loc[0, 'Stocks'])
top_2_str_to_list = ast.literal_eval(user_data_df.loc[1, 'Stocks'])
top_3_str_to_list = ast.literal_eval(user_data_df.loc[2, 'Stocks'])

# Get data from file for tickers in user_data using ".loc" and ".isin"
top_1_portfolio_returns = raw_stocks_data_df.loc[raw_stocks_data_df["Ticker"].isin(top_1_str_to_list)]
top_2_portfolio_returns = raw_stocks_data_df.loc[raw_stocks_data_df["Ticker"].isin(top_2_str_to_list)]
top_3_portfolio_returns = raw_stocks_data_df.loc[raw_stocks_data_df["Ticker"].isin(top_3_str_to_list)]

# Filter columns to get only the Ticker and Close Price and drop null values
top_1_portfolio_returns = top_1_portfolio_returns[["Ticker","Close_Price"]].dropna()
top_2_portfolio_returns = top_2_portfolio_returns[["Ticker","Close_Price"]].dropna()
top_3_portfolio_returns = top_3_portfolio_returns[["Ticker","Close_Price"]].dropna()

# Set tickers as columns
top_1_portfolio_returns = top_1_portfolio_returns.pivot_table("Close_Price","Date","Ticker")
top_2_portfolio_returns = top_2_portfolio_returns.pivot_table("Close_Price","Date","Ticker")
top_3_portfolio_returns = top_3_portfolio_returns.pivot_table("Close_Price","Date","Ticker")

# Calculate daily returns using "pct_change" and drop null values 
top_1_portfolio_returns  = top_1_portfolio_returns .pct_change().dropna() 
top_2_portfolio_returns  = top_2_portfolio_returns .pct_change().dropna() 
top_3_portfolio_returns  = top_3_portfolio_returns .pct_change().dropna() 

# Get User name to locate portfolios
top_1_user_name = user_data_df.loc[0, 'User_Name']
top_2_user_name = user_data_df.loc[1, 'User_Name']
top_3_user_name = user_data_df.loc[2, 'User_Name']

# Get the porfolio using weights and keep only the porfolio column
top_1_portfolio_returns[f"{top_1_user_name}"] = top_1_portfolio_returns.dot([1/len(top_1_str_to_list)] * len(top_1_str_to_list))
top_1_portfolio_returns = top_1_portfolio_returns[[f"{top_1_user_name}"]]

top_2_portfolio_returns[f"{top_2_user_name}"] = top_2_portfolio_returns.dot([1/len(top_2_str_to_list)] * len(top_2_str_to_list))
top_2_portfolio_returns = top_2_portfolio_returns[[f"{top_2_user_name}"]]

top_3_portfolio_returns[f"{top_3_user_name}"] = top_3_portfolio_returns.dot([1/len(top_3_str_to_list)] * len(top_3_str_to_list))
top_3_portfolio_returns = top_3_portfolio_returns[[f"{top_3_user_name}"]]

top_returns = pd.concat([top_1_portfolio_returns, top_2_portfolio_returns, top_3_portfolio_returns], axis="columns", join="inner")
top_returns.head()


Ticker,Test1,Amy,alex_1
Date,,,
2023-02-02,0.023946,0.017268,0.048580
2023-02-03,-0.007649,-0.009460,-0.027897
2023-02-06,-0.004281,-0.007270,-0.009067
2023-02-07,0.028546,0.025015,0.027996
2023-02-08,-0.008977,-0.018438,-0.009871


In [56]:
# Join market_return, top players with portfolio 
top_portfolio_market_returns = pd.concat([portfolio_returns["Portfolio"], market_returns, top_returns], axis="columns", join="inner")
top_portfolio_market_returns.head()

,Portfolio,SP500,NDAQ,Test1,Amy,alex_1
Date,,,,,,
2023-02-02,0.048580,0.014699,0.032540,0.023946,0.017268,0.048580
2023-02-03,-0.027897,-0.010355,-0.015889,-0.007649,-0.009460,-0.027897
2023-02-06,-0.009067,-0.006140,-0.009953,-0.004281,-0.007270,-0.009067
2023-02-07,0.027996,0.012873,0.019040,0.028546,0.025015,0.027996
2023-02-08,-0.009871,-0.011081,-0.016780,-0.008977,-0.018438,-0.009871


In [57]:
# Calculate sharp ratio - visulaize purposes - Portfolio and the Index
top_portfolio_market_sharp_ratio = (top_portfolio_market_returns.mean() * (252)) / (top_portfolio_market_returns.std() * np.sqrt(252))
top_portfolio_market_sharp_ratio

Portfolio    3.412431
SP500        1.739253
NDAQ         2.337676
Test1        3.526965
Amy          1.796389
alex_1       3.412431
dtype: float64

In [59]:
# Calculate cumulative returns to create plot
top_portfolio_market_stocks_cum_return = (1 + top_portfolio_market_returns).cumprod()
top_portfolio_market_stocks_cum_return.head()

,Portfolio,SP500,NDAQ,Test1,Amy,alex_1
Date,,,,,,
2023-02-02,1.048580,1.014699,1.032540,1.023946,1.017268,1.048580
2023-02-03,1.019328,1.004193,1.016134,1.016114,1.007644,1.019328
2023-02-06,1.010085,0.998026,1.006020,1.011764,1.000319,1.010085
2023-02-07,1.038364,1.010873,1.025175,1.040646,1.025342,1.038364
2023-02-08,1.028113,0.999672,1.007972,1.031303,1.006437,1.028113


# Visulization 